# Running Rank 1 models with OpenProblems data fromat

In this notebook, we will run rank 1 model with a dataset of OpenProblems data modality prediction task format. At 2022 competition, rank 1 model excelled in Multiome prediction task, so we'll focus on it. CITE-seq model leverages a lot of the same code and has a very similar architecture, so it should be possible to run it as well, just put CITE-seq data instead of Multiome in this tutorial and change task_type to "cite" in the scripts.

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
from scanpy.preprocessing._utils import _get_mean_var
import muon

/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Download 2021 competition data from s3://openproblems-data/resources_test/task_predict_modality/openproblems_neurips2021/

In [5]:
adata_rna_train = sc.read_h5ad("../data/openproblems_neurips2021/bmmc_multiome/normal/train_mod1.h5ad")
adata_rna_test = sc.read_h5ad("../data/openproblems_neurips2021/bmmc_multiome/normal/test_mod1.h5ad")
adata_atac_train = sc.read_h5ad("../data/openproblems_neurips2021/bmmc_multiome/normal/train_mod2.h5ad")
adata_atac_test = sc.read_h5ad("../data/openproblems_neurips2021/bmmc_multiome/normal/test_mod2.h5ad")

The dataset has both modalities, ATAC-seq and RNA in one AnnData object. Let's split modalities

In [6]:
adata_rna_train.shape, adata_rna_test.shape, adata_atac_train.shape, adata_atac_test.shape


((171, 1500), (427, 1500), (171, 1500), (427, 1500))

In [8]:
adata_atac = adata[:, adata.var["feature_types"] == "ATAC"]
adata_atac

View of AnnData object with n_obs × n_vars = 500 × 116490
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

Let's further reduce the number of features to 500 for RNA and 1000 for ATAC for faster testing.

In [40]:
def extract_day_and_donor_from_batch(batch: str):
    "Extract day and donor numbers N and M from IDs of format sNdM"
    d_index = batch.find("d")
    day = batch[1: d_index]
    donor = batch[d_index + 1 :]
    return day, donor

adata_rna_train.obs["day"], adata_rna_train.obs["donor"] = zip(*adata_rna_train.obs["batch"].astype(str).map(extract_day_and_donor_from_batch))
adata_rna_test.obs["day"], adata_rna_test.obs["donor"] = zip(*adata_rna_test.obs["batch"].astype(str).map(extract_day_and_donor_from_batch))
adata_atac_train.obs["day"], adata_atac_train.obs["donor"] = zip(*adata_atac_train.obs["batch"].astype(str).map(extract_day_and_donor_from_batch))
adata_atac_test.obs["day"], adata_atac_test.obs["donor"] = zip(*adata_atac_test.obs["batch"].astype(str).map(extract_day_and_donor_from_batch))

In [28]:
adata_atac_train.obs

,size_factors,batch,day,donor,split,DonorID
CGCCAAATCACCATTT-s2d4,241.0,s2d4,2,4,train,4
AGTGTGGCACCTGCCT-4-s2d1,259.0,s2d1,2,1,train,1
CGCCTGTGTATTCGCT-1-s1d1,165.0,s1d1,1,1,train,1
CACTGACCAGCTAACC-1-s1d1,157.0,s1d1,1,1,train,1
ATGTGAGAGGCTGGCT-s2d4,95.0,s2d4,2,4,train,4
...,...,...,...,...,...,...
ATTTAGCCACAACCTA-1-s1d1,281.0,s1d1,1,1,train,1
AGTTGGCGTGCCGCAA-s2d4,89.0,s2d4,2,4,train,4
ATGCAGGCAATTGACT-4-s2d1,240.0,s2d1,2,1,train,1
AGTCAAGAGGTCCTAG-4-s2d1,225.0,s2d1,2,1,train,1


Before modality prediction, apply QC to your dataset, otherwise the code will likely fail. Here, we'll simply remove cells with 0 RNA counts or open chromatin regions, and filter out constant features.

In [30]:
adata_rna_train.obs["split"] = "train"
adata_rna_test.obs["split"] = "test"
adata_atac_train.obs["split"] = "train"
adata_atac_test.obs["split"] = "test"

In [31]:
adata_rna = sc.concat([adata_rna_train, adata_rna_test], axis=0)
adata_atac = sc.concat([adata_atac_train, adata_atac_test], axis=0)

Put log-normalized data to `.X` layer of RNA and raw counts to `.X` of ATAC:

In [32]:
adata_rna.X = adata_rna.layers["normalized"]
adata_atac.X = adata_atac.layers["counts"]

In [33]:
non_empty_cells = (adata_rna.X.sum(axis=1) != 0) & (adata_atac.X.sum(axis=1) != 0)

In [34]:
adata_rna = adata_rna[non_empty_cells, :]
adata_atac = adata_atac[non_empty_cells, :]

In [35]:
rna_means, rna_vars = _get_mean_var(adata_rna.X)
adata_rna = adata_rna[:, rna_vars != 0]
adata_rna.shape

(598, 1500)

In [36]:
atac_means, atac_vars = _get_mean_var(adata_atac.X)
adata_atac = adata_atac[:, atac_vars != 0]
adata_atac.shape

(598, 1500)

Normalize ATAC data with TF-IDF. Note that RNA data must be library-size and log1p normalized as well.

In [37]:
muon.atac.pp.tfidf(adata_atac)
adata_atac.X[:10, :10].toarray()

/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/muon/_atac/preproc.py:84: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , 14.8241296 ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        

# Format the data correctly

To run the model, we need to save our data to the following files:
- test_multi_inputs.h5
- train_multi_inputs.h5
- train_multi_targets.h5

If you want to use CITE-seq model too, the you'll additionally need:
- test_cite_inputs
- train_cite_inputs
- train_cite_targets

Additionally, we must save test set labels to the "evaluation_ids.csv" file with "cell_id" column

In [ ]:
adata_rna.obs["donor"].value_counts()

DonorID
1     161
8      94
3      73
2      62
4      58
10     55
5      37
9      37
6      14
7       7
Name: count, dtype: int64

In [39]:
adata_rna[adata_rna.obs["split"] == "train"].write_h5ad("../data/train_multi_targets.h5")
adata_atac[adata_atac.obs["split"] == "train"].write_h5ad("../data/train_multi_inputs.h5")
adata_atac[adata_atac.obs["split"] == "test"].write_h5ad("../data/test_multi_inputs.h5")

/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/vladimir.shitov/minic

We also need to save metadata to a file. Note that in the current implementation, the metadata column **must** be named "day", "donor", "technology", "cell_type", and "cell_id". Day must be a number

In [20]:
adata_rna.obs.columns

Index(['GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors',
       'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments',
       'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction',
       'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order',
       'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality',
       'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType',
       'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker',
       'split'],
      dtype='object')

In [42]:
adata_rna.obs

,size_factors,batch,day,donor,split,DonorID
CGCCAAATCACCATTT-s2d4,241.0,s2d4,2,4,train,4
AGTGTGGCACCTGCCT-4-s2d1,259.0,s2d1,2,1,train,1
CGCCTGTGTATTCGCT-1-s1d1,165.0,s1d1,1,1,train,1
CACTGACCAGCTAACC-1-s1d1,157.0,s1d1,1,1,train,1
ATGTGAGAGGCTGGCT-s2d4,95.0,s2d4,2,4,train,4
...,...,...,...,...,...,...
TGTTTGTTCATCCTAT-2-s1d2,265.0,s1d2,1,2,test,2
CCCTCACCATTATGAC-14-s3d10,460.0,s3d10,3,10,test,10
AACCTTAAGCCAAATC-14-s4d8,22.0,s4d8,4,8,test,8
CCCAAATAGAACCTGT-14-s3d10,584.0,s3d10,3,10,test,10


In [43]:
adata_rna.obs["technology"] = "multiome"
adata_rna.obs.index.name = "cell_id"

/var/folders/p9/74j47tdn36n6s2vcv_5079q0s4wk4h/T/ipykernel_83981/1730522646.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_rna.obs["technology"] = "multiome"


In [47]:
adata_rna.obs["cell_type"] = "unknown"

In [48]:
adata_rna.obs[["day", "donor", "technology", "cell_type"]].reset_index(names="cell_id").to_csv("../data/metadata.csv")

Small function to save the data in the appropriate format:

In [51]:
def save_X_to_h5(adata, path):
    from pathlib import Path
    path = Path(path)
    X = adata.X.A if hasattr(adata.X, "A") else adata.X  # make dense if sparse
    df = pd.DataFrame(X, index=adata.obs_names, columns=adata.var_names).astype(np.float32)
    df.to_hdf(
        path,
        key=path.name,  # e.g. "train_multi_targets"
        mode="w",
        format="fixed",
    )

save_X_to_h5(adata_rna[adata_rna.obs["split"] == "train"], "../data/train_multi_targets.h5")
save_X_to_h5(adata_atac[adata_atac.obs["split"] == "train"], "../data/train_multi_inputs.h5")
save_X_to_h5(adata_atac[adata_atac.obs["split"] == "test"], "../data/test_multi_inputs.h5")

/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'train_multi_targets.h5'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'train_multi_inputs.h5'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/vladimir.shitov/miniconda3/envs/2022_12_kaggle_ablation_study/lib/python3.8/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 't

Run scripts as recommended by competitor. For the given subset of data, it takes ~30 minutes on Macbook (cpu mode). If you wish to use CITE-seq model, change `task_type` to `cite` in the last script. You might also want to download additional files with biological priors for CITE-seq prediction. See [readme](https://github.com/lueckenlab/OpenProblems2022Analysis/tree/main/code/rank1/open-problems-multimodal) of the original repository for details

In [52]:
%%bash

export DATA_DIR=/Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/

cd rank1/open-problems-multimodal/
python3 script/make_compressed_dataset.py --data_dir ${DATA_DIR}
python3 script/make_additional_files.py --data_dir ${DATA_DIR}
python3 script/make_compressed_dataset.py --data_dir ${DATA_DIR}
python3 script/train_model.py --data_dir ${DATA_DIR} --task_type multi 

File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/evaluation_ids.csv does not exist
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/sample_submission.csv does not exist
427
171
171
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/test_cite_inputs.h5 does not exist
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/train_cite_inputs.h5 does not exist
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/train_cite_targets.h5 does not exist
Some citeseq files don't exist, not making citeseq cell statistics
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/evaluation_ids.csv does not exist
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/sample_submission.csv does not exist
427
171
171
File /Users/vladimir.shitov/Documents/programming/OpenProblems2022Analysis/data/test_cite_inputs.h5

The results are saved as pickled numpy array. Let's read them and calculate a correlation score to ground truth. We can import a function to compute the metric directly from competitor's code:

In [53]:
import sys
sys.path.append("rank1/open-problems-multimodal")  # Make it possible to import from competitor's code

from ss_opm.metric.correlation_score import correlation_score

In [54]:
import pickle

with open("rank1/open-problems-multimodal/result/multimodal_pred.pickle", "rb") as f:
    y_pred = pickle.load(f)

In [55]:
y_pred.shape

(427, 1500)

In [56]:
y_pred

array([[ 0.48803106, -0.65492237, -0.6615395 , ...,  0.50882393,
        -0.6830433 , -0.69751734],
       [ 0.0964658 , -0.6530562 , -0.5683277 , ...,  0.989331  ,
        -0.04067516, -0.47889504],
       [ 0.6118262 , -0.7402453 , -0.42986473, ..., -0.02475481,
         0.09570294, -0.4786224 ],
       ...,
       [ 0.32633352, -0.5289058 , -0.60409075, ...,  1.2354933 ,
        -0.42940715, -0.5399292 ],
       [ 0.54016536, -0.5665478 , -0.583822  , ...,  1.350588  ,
        -0.02383605, -0.25707066],
       [-0.1975743 , -0.556948  , -0.6063245 , ...,  0.92392415,
        -0.6779063 , -0.6611087 ]], dtype=float32)

In [57]:
y_true = adata_rna[adata_rna.obs["split"] == "test"].X
correlation_score(y_true, y_pred)

0.3650121607214872